In [3]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

In [17]:
X = np.random.randint(0, 10, 100)
y = np.random.randint(0, 10, 100)

In [26]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)

In [27]:
splits = skf.split(X, y)

In [28]:
for i, (train_index, test_index) in enumerate(splits):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

Fold 0:
  Train: index=[ 4  5  6  7  8  9 13 14 15 17 18 19 20 21 22 23 25 26 29 30 31 32 33 36
 37 39 40 42 45 46 48 49 51 52 53 55 57 59 61 65 66 67 68 70 71 72 73 74
 75 76 78 79 81 83 85 86 88 89 90 91 92 93 94 96 97 99]
  Test:  index=[ 0  1  2  3 10 11 12 16 24 27 28 34 35 38 41 43 44 47 50 54 56 58 60 62
 63 64 69 77 80 82 84 87 95 98]
Fold 1:
  Train: index=[ 0  1  2  3  5  6  8  9 10 11 12 15 16 17 19 20 21 24 26 27 28 30 31 33
 34 35 38 41 42 43 44 47 50 51 54 55 56 58 59 60 61 62 63 64 65 66 67 68
 69 70 73 74 76 77 80 81 82 84 85 86 87 94 95 96 97 98 99]
  Test:  index=[ 4  7 13 14 18 22 23 25 29 32 36 37 39 40 45 46 48 49 52 53 57 71 72 75
 78 79 83 88 89 90 91 92 93]
Fold 2:
  Train: index=[ 0  1  2  3  4  7 10 11 12 13 14 16 18 22 23 24 25 27 28 29 32 34 35 36
 37 38 39 40 41 43 44 45 46 47 48 49 50 52 53 54 56 57 58 60 62 63 64 69
 71 72 75 77 78 79 80 82 83 84 87 88 89 90 91 92 93 95 98]
  Test:  index=[ 5  6  8  9 15 17 19 20 21 26 30 31 33 42 51 55 59 61 65 66 67 68 

In [29]:
from hydroDL import kPath
from hydroDL.data import dbVeg, DataModel
from hydroDL.master import basinFull, slurm, dataTs2Range

import numpy as np

import pandas as pd
import pickle
import os


loading package hydroDL


In [34]:
dataName = "singleDaily"
rho = 45

In [35]:
df = dbVeg.DataFrameVeg(dataName)
dm = DataModel(X=df.x, XC=df.xc, Y=df.y)
siteIdLst = df.siteIdLst
dm.trans(mtdDefault='minmax')
dataTup = dm.getData()
dataEnd, (iInd, jInd) = dataTs2Range(dataTup, rho, returnInd=True)
x, xc, y, yc = dataEnd

In [37]:
varS = ['VV', 'VH', 'vh_vv']
varL = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'ndvi', 'ndwi', 'nirv']
varM = ['Fpar', 'Lai']
iS = [df.varX.index(var) for var in varS]
iL = [df.varX.index(var) for var in varL]
iM = [df.varX.index(var) for var in varM]

In [39]:
pSLst, pLLst, pMLst = list(), list(), list()
ns = yc.shape[0]
nMat = np.zeros([yc.shape[0], 3])
for k in range(nMat.shape[0]):
    tempS = x[:, k, iS]
    pS = np.where(~np.isnan(tempS).any(axis=1))[0]
    tempL = x[:, k, iL]
    pL = np.where(~np.isnan(tempL).any(axis=1))[0]
    tempM = x[:, k, iM]
    pM = np.where(~np.isnan(tempM).any(axis=1))[0]
    pSLst.append(pS)
    pLLst.append(pL)
    pMLst.append(pM)
    nMat[k, :] = [len(pS), len(pL), len(pM)]

In [40]:
indKeep = np.where((nMat > 0).all(axis=1))[0]
x = x[:, indKeep, :]
xc = xc[indKeep, :]
yc = yc[indKeep, :]
nMat = nMat[indKeep, :]
pSLst = [pSLst[k] for k in indKeep]
pLLst = [pLLst[k] for k in indKeep]
pMLst = [pMLst[k] for k in indKeep]
jInd = jInd[indKeep]
siteIdLst = [siteIdLst[k] for k in jInd]

In [47]:
jSite, count = np.unique(jInd, return_counts=True)
countAry = np.array([[x, y] for y, x in sorted(zip(count, jSite))])
nRm = sum(countAry[:, 1] < 5)
indSiteAll = countAry[nRm:, 0].astype(int)
dictSubset = dict()

In [138]:
lcIdx = [df.varXC.index(f"lc{i}") for i in range(1, 10)]
lc_pct = df.xc[:, lcIdx]
top_lcs = np.argmax(lc_pct, axis=1)
site_lcs = pd.DataFrame({"lc": top_lcs, "site": df.siteIdLst})
site_lcs = site_lcs[site_lcs.index.isin(indSiteAll)]

In [139]:
site_lcs

,lc,site
0,4,N0005
1,4,N0013
2,0,N0015
3,4,N0016
4,4,N0019
...,...,...
324,3,N1102
325,7,N1103
326,6,N1104
327,3,N1105


In [153]:
skf = StratifiedKFold(n_splits=3)

In [154]:
splits = skf.split(site_lcs.site, site_lcs.lc)

In [155]:
dictSubset = {}
for k, (train_indices, test_indices) in enumerate(splits):
    siteTrain= [site_lcs.index[index] for index in train_indices] 
    siteTest= [site_lcs.index[index] for index in test_indices] 
    indTest = np.where(np.isin(jInd, siteTest))[0]
    indTrain = np.where(np.isin(jInd, siteTrain))[0]
    dictSubset['testSite_k{}'.format(k)] = siteTest
    dictSubset['trainSite_k{}'.format(k)] = siteTrain
    dictSubset['testInd_k{}'.format(k)] = indTest.tolist()
    dictSubset['trainInd_k{}'.format(k)] = indTrain.tolist()

/Users/andyhuynh/anaconda3/envs/pytorch/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


In [161]:
data_tuple = (df, dictSubset['trainInd_k0'], dictSubset['testInd_k0'], nMat, pSLst, pLLst, pMLst, x, xc, yc) 

In [162]:
# data_tuple = (df, dictSubset, nMat, pSLst, pLLst, pMLst, x, xc, yc) 

with open(os.path.join(kPath.dirVeg, 'stratified_3_fold.pkl'), 'wb') as f:  # open a text file
    pickle.dump(data_tuple, f) # serialize the list